In [1]:
import torch
import matplotlib.pyplot as plt
import os
from IPython.display import Audio

from src.train import train
from src.NSynthDataset import NSynthDataset
from src.utils import estimate_pitch

In [2]:
batch_size = 50
D_lr = 1e-5
G_lr = 1e-4
epochs = 300
z_size = 1000

# min len =16000
sr = 8192
duration = 2

In [3]:
os.chdir('../../..')
path = 'mnt/data/public/NSynth/'

In [4]:
train_set = NSynthDataset(data_path=path, 
                          mel=False,  pitched_z=False,
                          sampling_rate=sr, duration=duration, 
                          min_class_count=10000, max_class_count=3000,
                          z_size=z_size)

In [5]:
i = 42
w, l, z = train_set.__getitem__(i)
inst = train_set.annot.loc[train_set.fnames[i], 'instrument_class_str']
pitch = train_set.annot.loc[train_set.fnames[i], 'pitch']
print(inst, pitch)

organ_electronic 80


In [6]:
Audio(w, rate=sr)

In [7]:
g_loss, d_loss, G, D = train(train_set, 
                             batch_size, 
                             D_lr, 
                             G_lr, 
                             epochs, 
                             z_size, 
                             d_updates=5, 
                             g_updates=1, 
                             flip_prob=0.05, 
                             ph=2, 
                             loss='wasserstein', 
                             save_dir='users/adcy353/GANs-Conditional-Audio-Synthesis/models/wavegan unpitched/',
                             pretr_epochs=977,
                             save_epochs=25
                            )#, val_set=val_set)


Working on cuda
Loading data...
Creating WaveGAN...
WaveGANGenerator(
  (fc): Linear(in_features=1006, out_features=16384, bias=True)
  (deconv): Sequential(
    (0): ConvTranspose1d(1024, 512, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (1): ReLU()
    (2): ConvTranspose1d(512, 256, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (3): ReLU()
    (4): ConvTranspose1d(256, 128, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (5): ReLU()
    (6): ConvTranspose1d(128, 64, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (7): ReLU()
    (8): ConvTranspose1d(64, 1, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (9): Tanh()
  )
)
WaveGANDiscriminator(
  (lrelu): LeakyReLU(negative_slope=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (conv_layers): Sequential(
    (0): Conv1d(1, 64, kernel_size=(25,), stride=(4,), padding=(11,))
    (1): Conv1d(64, 128, kernel_size=(2

100%|██████████| 300/300 [02:54<00:00,  1.72it/s]


EPOCH 1/300: Generator loss: 11.5, Discriminator loss: -29.0, Train accuracy: 0.66213


total_memory: 85198045184, available_memory: 2016852992


100%|██████████| 300/300 [02:50<00:00,  1.76it/s]

...

100%|██████████| 300/300 [02:54<00:00,  1.72it/s]


EPOCH 9/300: Generator loss: -7.3, Discriminator loss: -29.5, Train accuracy: 0.65210


total_memory: 85198045184, available_memory: 2016852992


 24%|██▍       | 72/300 [00:35<01:56,  1.96it/s]/users/adcy353/GANs-Conditional-Audio-Synthesis/src/NSynthDataset.py:80: UserWarning: PySoundFile failed. Trying audioread instead.
/users/adcy353/.local/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
 24%|██▍       | 72/300 [00:36<01:54,  1.99it/s]


OSError: [Errno 12] Cannot allocate memory

In [ ]:
with open(f'{save_dir}loss_hist.txt', 'w') as f:
    f.write(f'G: {g_loss}\n D: {d_loss}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
ax.plot(range(epochs), d_loss, label='Discriminator loss')
ax.plot(range(epochs), g_loss, label='Generator loss')
ax.legend()

In [ ]:
G.eval()
s = G.forward(z.unsqueeze(0).to(torch.device('cuda')), l.unsqueeze(0).to(torch.device('cuda')))
s.to(torch.device('cpu'))
s = s.detach().cpu()
Audio(s, rate=sr)

In [ ]:
estimate_pitch(s, sr)

1e-3, 0, 25, no sch -> gdom